* Based on https://curly-space-meme-rjgqp54j94vhp5x7.github.dev/ 
* To set up, run: mamba activate oggm_env
* Using oggm_env (Python 3.12.2) kernel 

In [6]:
# basic
import os 
import psutil
import pandas as pd
import numpy as np
import glob as glob
import xarray_regrid # type: ignore
# using regrid instead of tqdm because was having a lot of issues trying to import 
from datetime import datetime

# geospatial
import xarray as xr
import geopandas as gpd

# oggm
from oggm import cfg, utils, workflow, tasks
from oggm.shop import gcm_climate

## Setup and possible options 

first line was initially 
# cfg.initialize(logging_level='WORKFLOW', future = True)
but I deleted future because apparently that's not an accepted function? 

In [7]:
cfg.initialize(logging_level='WORKFLOW')

cfg.PARAMS['use_multiprocessing']  = True
cfg.PARAMS['baseline_climate']     = ''
cfg.PARAMS['prcp_scaling_factor']  = 1
cfg.PARAMS['hydro_month_sh']       = 1
cfg.PARAMS['hydro_month_nh']       = 1
cfg.PARAMS['border']               = 80
cfg.PARAMS['min_mu_star']          = 5
cfg.PARAMS['max_mu_star']          = 800
cfg.PARAMS['geodetic_mb_period']   = '2000-01-01_2020-01-01' 
cfg.PARAMS['store_model_geometry'] = True
cfg.PARAMS['continue_on_error']    = True
cfg.PARAMS['use_winter_prcp_factor'] = False

# Potential historical scenario
outlines    = ["RGI7"]      # Glacier outlines
climate_ds  = ["PMET", "CR2MET", "MSWEP", "ERA5"]  # Climate baseline
volume_ds   = ["M22", "F19"]            # Reference volume dataset 

# Potential future scenario
gcm_list  = ["ACCESS-CM2", "BCC-CSM2-MR", "CMCC-ESM2", "FGOALS-f3-L", "GFDL-ESM4", "CMCC-CM2-SR5", "KACE-1-0-G", "MPI-ESM1-2-HR", "MRI-ESM2-0", "MIROC6"] # Climate models
ssp_list  = ["ssp126","ssp245", "ssp370", "ssp585"]   # Future scenarios
bias_correction = ["MVA", "DQM", "MBC"]              # Bias correction method

2024-07-25 18:34:46: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2024-07-25 18:34:46: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2024-07-25 18:34:46: oggm.cfg: Multiprocessing: using all available processors (N=2)
2024-07-25 18:34:48: oggm.cfg: Multiprocessing switched ON after user settings.
2024-07-25 18:34:48: oggm.cfg: PARAMS['baseline_climate'] changed from `GSWP3_W5E5` to ``.
2024-07-25 18:34:48: oggm.cfg: WARNING: adding an unknown parameter `prcp_scaling_factor`:`1` to PARAMS.
2024-07-25 18:34:48: oggm.cfg: PARAMS['hydro_month_sh'] changed from `4` to `1`.
2024-07-25 18:34:48: oggm.cfg: PARAMS['hydro_month_nh'] changed from `10` to `1`.
2024-07-25 18:34:48: oggm.cfg: WARNING: adding an unknown parameter `min_mu_star`:`5` to PARAMS.
2024-07-25 18:34:48: oggm.cfg: WARNING: adding an unknown parameter `max_mu_star`:`800` to PARAMS.
2024-07-25 18:34:48: oggm.cfg: PARAMS['store_model_geometry'] changed from `False`

## Calibration in the reference period (and commitment run)
The complete workflow was divided according to tasks due to RAM limitations (only 32)

In [8]:
for rgi in outlines:
    for file_id in climate_ds: 
        for volume in volume_ds:
            
            start = datetime.now()
            
            # ids = gpd.read_file("/home/hydro/Dropbox/Patagonia/GIS South/Glaciers/" + rgi + "_v2.shp")
            ids = ['RGI60-17.13009', 'RGI60-17.13011', 'RGI60-17.13013', 'RGI60-17.13014', 'RGI60-17.13018', 'RGI60-17.13020', 'RGI60-17.13021', 
                   'RGI60-17.13022', 'RGI60-17.13024', 'RGI60-17.13026', 'RGI60-17.13029', 'RGI60-17.13030', 'RGI60-17.13031', 'RGI60-17.13033', 
                   'RGI60-17.13034', 'RGI60-17.13044', 'RGI60-17.13045', 'RGI60-17.13047', 'RGI60-17.13049', 'RGI60-17.13050', 'RGI60-17.13051',
                   'RGI60-17.13052', 'RGI60-17.13054', 'RGI60-17.13056', 'RGI60-17.13058'] 
            # had a command to only look at big glaciers but that doesn't serve in my case 
            
            # cfg.PATHS['working_dir']  = "/home/hydro/OGGM_results/" + rgi + "_" + file_id + "_" + volume +"_run"
            cfg.PATHS['working_dir']  = "/workspaces/oggm_redo/oggm" + rgi + "_" + file_id + "_" + volume +"_run"
            cfg.PATHS['climate_file'] = "/home/hydro/OGGM_results/" + file_id + "_OGGM_1980_2019m.nc"

            if rgi == "RGI7":
                cfg.PARAMS['use_rgi_area'] = False
                cfg.PARAMS['use_intersects'] = False
                cfg.PARAMS['rgi_version'] = 7
                
                columns_ids = {'glac_id': 'GLIMSId', 'area_km2' : 'Area', 'glac_name': 'Name', 'Zone' : 'Zone', 
                               'vol_F19': 'vol_F19', 'vol_M22':'vol_M22', 'dmdtda_21' :'dmdtda_21'}
                ids = utils.cook_rgidf(ids, o1_region='17', o2_region='02', bgndate= ids.src_date, version = "70",
                              assign_column_values= columns_ids)
                 
                gdirs = workflow.init_glacier_directories(ids)
                workflow.execute_entity_task(tasks.define_glacier_region, gdirs, source="NASADEM");

                task_list = [tasks.process_dem, tasks.simple_glacier_masks, tasks.elevation_band_flowline,
                             tasks.fixed_dx_elevation_band_flowline, tasks.compute_downstream_line, tasks.compute_downstream_bedshape]
             
                for task in task_list:
                    workflow.execute_entity_task(task, gdirs);

            # write climate file for each glacier
            workflow.execute_entity_task(tasks.process_custom_climate_data, gdirs);
            
            # calibration using huggonet et al. 2021            
            ref_mb21 = ids.set_index("RGIId").dmdtda_21*1000    
            workflow.execute_entity_task(tasks.mu_star_calibration_from_geodetic_mb, 
                                             [(gdir, {'ref_mb': float(ref_mb21.loc[gdir.rgi_id])}) for gdir in gdirs]);
            
            # subset gdirs to avoid infinitive error messages
            workflow.execute_entity_task(tasks.apparent_mb_from_any_mb, gdirs);
            df = utils.compile_task_log(gdirs, task_names=["apparent_mb_from_any_mb"])
            df = df[df.apparent_mb_from_any_mb == "SUCCESS"]
            gdirs = [gdir for gdir in gdirs if gdir.rgi_id in df.index.tolist()]

            # inversion by catchment
            for zone in range(1,10):
                ids_subset = ids[ids.Zone == zone]
                gdirs_subset = [gdir for gdir in gdirs if gdir.rgi_id in ids_subset.RGIId.tolist()]

                if volume == "M22": # Millan et al. 2022 
                    workflow.calibrate_inversion_from_consensus(gdirs_subset, volume_m3_reference = ids_subset.vol_M22.sum()*1e9,
                                                                apply_fs_on_mismatch=True, error_on_mismatch=False, 
                                                                filter_inversion_output=True);
                else: # Farinotti et al. 2019
                    workflow.calibrate_inversion_from_consensus(gdirs_subset, volume_m3_reference = ids_subset.vol_F19.sum()*1e9,
                                                                apply_fs_on_mismatch=True, error_on_mismatch=False, 
                                                                filter_inversion_output=True);
            # ready to use (calibrate)
            workflow.execute_entity_task(tasks.init_present_time_glacier, gdirs); 

            # calibration
            workflow.execute_entity_task(tasks.run_with_hydro, 
                             [(gdir, {'ref_dmdtda': float(ref_mb21.loc[gdir.rgi_id])}) for gdir in gdirs], 
                             run_task = tasks.run_dynamic_mu_star_calibration, 
                             store_monthly_hydro=True, 
                             ys=1980,
                             ye=2020,
                             ref_area_from_y0=True,
                             output_filesuffix= "_" + file_id,
                             err_ref_dmdtda = 0.25*1000,
                             mu_star_max_step_length = 10, 
                             maxiter = 20,         
                             ignore_errors=True);

            utils.compile_glacier_statistics(gdirs);  # save results
            utils.compile_run_output(gdirs, input_filesuffix= "_" + file_id)

            ## commitment run
            workflow.execute_entity_task(tasks.run_with_hydro, gdirs,
                                                 run_task = tasks.run_random_climate,
                                                 seed = 123,
                                                 nyears = 80, 
                                                 y0 = 2000, # central year of the random climate period. Has to be set!
                                                 halfsize = 15, 
                                                 store_monthly_hydro = True, 
                                                 output_filesuffix = '_ct_random',
                                                 init_model_filesuffix = "_" + file_id,  # this is important! Start from 2020 glacier
                                                 ref_geometry_filesuffix = "_" + file_id,  # also use this as area reference
                                                 ref_area_from_y0 = True)  # and keep the same reference area as for the hist simulations
            utils.compile_run_output(gdirs, input_filesuffix='_ct_random')

2024-07-25 18:34:54: oggm.cfg: PARAMS['use_rgi_area'] changed from `True` to `False`.
2024-07-25 18:34:54: oggm.cfg: PARAMS['use_intersects'] changed from `True` to `False`.
2024-07-25 18:34:54: oggm.cfg: PARAMS['rgi_version'] changed from `62` to `7`.


AttributeError: 'list' object has no attribute 'src_date'

## Climate projections for each glacier and scenario

In [ ]:
for rgi in outlines:
    for file_id in climate_ds: 
        for volume in volume_ds:

            ids = gpd.read_file("/home/hydro/Dropbox/Patagonia/GIS South/Glaciers/" + rgi + "_v2.shp")
            ids = ids[ids.area_km2 > 1] # ~ 2,000 glaciers in each RGI
            
            cfg.PATHS['working_dir']  = "/home/hydro/OGGM_results/" + rgi + "_" + file_id + "_" + volume +"_run"
            cfg.PATHS['climate_file'] = "/home/hydro/OGGM_results/" + file_id + "_OGGM_1980_2019m.nc"

            if rgi == "RGI6":
                # init directories
                cfg.PARAMS['use_rgi_area'] = True
                cfg.PARAMS['use_intersects'] = True
                cfg.PARAMS['rgi_version'] = 62

            if rgi == "RGI7":
                cfg.PARAMS['use_rgi_area'] = False
                cfg.PARAMS['use_intersects'] = False
                cfg.PARAMS['rgi_version'] = 7
                
                columns_ids = {'glac_id': 'GLIMSId', 'area_km2' : 'Area', 'glac_name': 'Name', 'Zone' : 'Zone', 
                               'vol_F19': 'vol_F19', 'vol_M22':'vol_M22', 'dmdtda_21' :'dmdtda_21'}
                ids = utils.cook_rgidf(ids, o1_region='17', o2_region='02', bgndate= ids.src_date, version = "70",
                              assign_column_values= columns_ids)
                 
            gdirs = workflow.init_glacier_directories(ids)

            # future projections
            for gcm in tqdm(gcm_list):    
                for ssp in tqdm(ssp_list, leave = False):
                    for bc in bias_correction:
                        rid = "_{}_{}_{}".format(gcm, ssp, bc)

                        # write future climate file (SSP-based) for each glacier
                        workflow.execute_entity_task(gcm_climate.process_cmip_data, gdirs, filesuffix =  rid, 
                                                 fpath_precip = "/home/hydro/OGGM_results/Future_climate_bc/PP_" + file_id + rid + ".nc", 
                                                 fpath_temp = "/home/hydro/OGGM_results/Future_climate_bc/T2M_" + file_id + rid + ".nc", 
                                                 apply_bias_correction=False);

## Glacier projections for each glacier and scenario

In [ ]:
for rgi in outlines:
    for file_id in climate_ds: 
        for volume in volume_ds:

            ids = gpd.read_file("/home/hydro/Dropbox/Patagonia/GIS South/Glaciers/" + rgi + "_v2.shp")
            ids = ids[ids.area_km2 > 1] # ~ 2,000 glaciers in each RGI
            
            cfg.PATHS['working_dir']  = "/home/hydro/OGGM_results/" + rgi + "_" + file_id + "_" + volume +"_run"
            cfg.PATHS['climate_file'] = "/home/hydro/OGGM_results/" + file_id + "_OGGM_1980_2019m.nc"

            if rgi == "RGI6":
                # init directories
                cfg.PARAMS['use_rgi_area'] = True
                cfg.PARAMS['use_intersects'] = True
                cfg.PARAMS['rgi_version'] = 62

            if rgi == "RGI7":
                cfg.PARAMS['use_rgi_area'] = False
                cfg.PARAMS['use_intersects'] = False
                cfg.PARAMS['rgi_version'] = 7
                
                columns_ids = {'glac_id': 'GLIMSId', 'area_km2' : 'Area', 'glac_name': 'Name', 'Zone' : 'Zone', 
                               'vol_F19': 'vol_F19', 'vol_M22':'vol_M22', 'dmdtda_21' :'dmdtda_21'}
                ids = utils.cook_rgidf(ids, o1_region='17', o2_region='02', bgndate= ids.src_date, version = "70",
                              assign_column_values= columns_ids)
                 
            gdirs = workflow.init_glacier_directories(ids)

            # subset gdirs to avoid infinitive error messages
            df = pd.read_csv(cfg.PATHS['working_dir'] + "/task_log.csv")
            df = df[df.apparent_mb_from_any_mb == "SUCCESS"]
            gdirs = [gdir for gdir in gdirs if gdir.rgi_id in df.rgi_id.tolist()]
                
            for gcm in tqdm(gcm_list):    
                for ssp in tqdm(ssp_list):
                    for bc in bias_correction:
                        rid = "_{}_{}_{}".format(gcm, ssp, bc)

                        # run the glacier using hydro function 
                        workflow.execute_entity_task(tasks.run_with_hydro, gdirs, run_task = tasks.run_from_climate_data,
                                                 climate_filename = 'gcm_data',  # use gcm_data, not climate_historical
                                                 climate_input_filesuffix = rid,  # use the chosen scenario
                                                 init_model_filesuffix = "_" + file_id,  # this is important! Start from 2020 glacier
                                                 ref_geometry_filesuffix = "_" + file_id,  # also use this as area reference
                                                 ref_area_from_y0 = True,  # and keep the same reference area as for the hist simulations
                                                 output_filesuffix = rid,  # recognize the run for later
                                                 store_monthly_hydro = True)  # add monthly diagnostics
                        utils.compile_run_output(gdirs, input_filesuffix=rid)

                    print(rgi, file_id, volume, gcm, ssp, datetime.now()-start, "RAM memory % used:", psutil.virtual_memory()[2])